In [1]:
import numpy as np                   # Data manipulation
import pandas as pd                  # DataFrame manipulation
import time                          # To time processes 
import warnings                      # To suppress warnings
import matplotlib.pyplot as plt      # For Graphics
import seaborn as sns
from sklearn import cluster, mixture # For clustering 
from sklearn.preprocessing import StandardScaler

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
file = "happiness.csv"
file_pd = pd.read_csv(file)
file_pd.head()

,Unnamed: 0,Country Name,Happiness Rank 2015,Happiness Score 2015,Happiness Rank 2016,Happiness Score 2016,GDP(billions) 2015,"GDP, 2016 (billions)",Total Population 2015,Total Population 2016,GDP/Pop 2015,GDP/Pop 2016,GDI 2015,GDI 2016,GDI/Pop 2015,GDI/Pop 2016,CPI 2015,CPI 2016
0,0,Switzerland,1,7.587,2,7.509,679.00,669.0,8282.40,8372.41,81.981068,79.905308,6.673750e+11,6.656087e+11,8.057749e+07,7.950025e+07,100.617575,100.180283
1,1,Iceland,2,7.561,3,7.501,16.94,20.3,330.82,335.44,51.206094,60.517529,1.891027e+12,2.063429e+12,5.716182e+09,6.151410e+09,239.815158,243.876225
2,2,Denmark,3,7.527,1,7.526,301.00,307.0,5683.48,5728.01,52.960510,53.596275,1.928783e+12,1.958050e+12,3.393665e+08,3.418377e+08,100.000000,100.250000
3,3,Norway,4,7.522,4,7.498,387.00,371.0,5190.24,5236.15,74.563026,70.853585,2.739875e+12,2.682282e+12,5.278899e+08,5.122622e+08,100.000000,103.550000
4,4,Canada,5,7.427,6,7.404,1560.00,1540.0,35832.51,36264.60,43.535884,42.465655,1.819117e+12,1.834327e+12,5.076722e+07,5.058176e+07,126.566667,128.375000


In [3]:
file_pd=file_pd.fillna(value=0)

In [23]:
data = dict(type = 'choropleth', 
           locations = file_pd['Country Name'],
           locationmode = 'country names',
           z = file_pd['Happiness Score 2015'], 
           text = file_pd['Country Name'],
           colorbar = {'title':'Happiness Index'})
layout = dict(title = 'Happiness Visualization', 
             geo = dict(showframe = False, 
                       projection = {'type': 'Mercator'}))
choromap3 = go.Figure(data = [data], layout=layout)
iplot(choromap3)